In [1]:
from io import BytesIO

from bitcoin_client.ledger_bitcoin.common import hash160
from bitcoin_client.ledger_bitcoin.psbt import PSBT, KeyOriginInfo, PartiallySignedInput
from bitcoin_client.ledger_bitcoin.tx import CTransaction, CTxWitness, CTxInWitness, COutPoint, uint256_from_str, CTxIn, CTxOut
from embit.descriptor import Descriptor, Key
from embit.descriptor.miniscript import Miniscript
from embit.networks import NETWORKS
from bip32 import BIP32
from mnemonic import Mnemonic
mnemo = Mnemonic("english")

DEFAULT_SPECULOS_MNEMONIC = "glory promote mansion idle axis finger extra february uncover one trip resource lawn turtle enact monster seven myth punch hobby comfort wild raise skin"
seed = mnemo.to_seed(DEFAULT_SPECULOS_MNEMONIC)
bip32 = BIP32.from_seed(seed, network="test")

master_extended_privkey = bip32.get_xpriv()
master_extended_pubkey = bip32.get_xpub()
fpr = hash160(bip32.pubkey)[0:4]
H = 0x80000000
fpr.hex()

'f5acc2fd'

In [2]:
PSBT_IMPLICIT_ALL="cHNidP8BAH0CAAAAAaG4I9IzbWlLSTTvm25bfeF6BVE9qKKdsCouy8eppv5tAQAAAAD9////AlX/pwAAAAAAIlEgC450hrwwagrvt6fACvBAVULbGs1z7syoJ3HM9f5etg+ghgEAAAAAABYAFBOZuKCYR6A5sDUvWNISwYC6sX93AAAAAAABASunhqkAAAAAACJRINj08dGJltthuxyvVCPeJdih7unJUNN+b/oCMBLV5i4NIRYhLqKFalzxEOZqK+nXNTFHk/28s4iyuPE/K2remC569RkA9azC/VYAAIABAACAAAAAgAEAAAAAAAAAARcgIS6ihWpc8RDmaivp1zUxR5P9vLOIsrjxPytq3pguevUAAQUgApCB7OVhaqHLmTGfxIdO/uR/CM66X2AEY2yMQ0CaXwohBwKQgezlYWqhy5kxn8SHTv7kfwjOul9gBGNsjENAml8KGQD1rML9VgAAgAEAAIAAAACAAQAAAAIAAAAAAA=="


In [3]:
# SIGHASH_ALL_SIGN
psbt = PSBT()
psbt.deserialize(PSBT_IMPLICIT_ALL)

# input 0 
input_pk0 = bip32.get_pubkey_from_path("m/86'/1'/0'/1/2")
input_pk0 = input_pk0[1:] 

psbt.tx.vin[0].prevout.hash = int("0924ec49c92796d5dfc3adf35e838e83d86cffe8e2c6543ce837ec63319af716", 16)
psbt.tx.vin[0].prevout.n = 0
psbt.tx.vin[0].nSequence = 4294967293
psbt.tx.vout[0].nValue = 9909202
psbt.inputs[0].sighash = 1
psbt.inputs[0].witness_utxo.nValue = 9909389
psbt.inputs[0].witness_utxo.scriptPubKey = bytes.fromhex("51200b8e7486bc306a0aefb7a7c00af0405542db1acd73eecca82771ccf5fe5eb60f")
psbt.inputs[0].tap_internal_key=bytes.fromhex("029081ece5616aa1cb99319fc4874efee47f08ceba5f6004636c8c43409a5f0a")
psbt.inputs[0].tap_bip32_paths = {input_pk0: (set(), KeyOriginInfo(fpr, [86^H, 1^H, 0^H, 1, 2]))}

psbt.tx.vin.append(CTxIn())
psbt.inputs.append(PartiallySignedInput(0))

#input 1
psbt.inputs[1].witness_utxo = CTxOut()

input_pk1 = bip32.get_pubkey_from_path("m/86'/1'/0'/0/2")
input_pk1 = input_pk1[1:] 

psbt.tx.vin[1].prevout.hash = int("b3c0eacacd8a1db1fbb76b4c2854281291031c9403fd542286e1817c871be2f9", 16)
psbt.tx.vin[1].prevout.n = 0
psbt.tx.vin[1].nSequence = 4294967293
psbt.tx.vout[1].nValue = 9813
psbt.inputs[1].sighash = 1
psbt.inputs[1].witness_utxo.nValue = 10000
psbt.inputs[1].witness_utxo.scriptPubKey = bytes.fromhex("512041461489a43cb67e2c64f39427278edd30c03d6d72a4b89986147c428f0cc5f1")
psbt.inputs[1].tap_internal_key=bytes.fromhex("eaf7653443bec65437dba1da52f9ca97e231785af56627e8f9c311cf7300f6de")
psbt.inputs[1].tap_bip32_paths = {input_pk1: (set(), KeyOriginInfo(fpr, [86^H, 1^H, 0^H, 0, 2]))}

psbt.inputs[1].witness_utxo.scriptPubKey.hex()

NEW_PSBT = psbt.serialize()
print(NEW_PSBT)

cHNidP8BAKYCAAAAAhb3mjFj7DfoPFTG4uj/bNiDjoNe863D39WWJ8lJ7CQJAAAAAAD9////+eIbh3yB4YYiVP0DlBwDkRIoVChMa7f7sR2KzcrqwLMAAAAAAP3///8C0jOXAAAAAAAiUSALjnSGvDBqCu+3p8AK8EBVQtsazXPuzKgnccz1/l62D1UmAAAAAAAAFgAUE5m4oJhHoDmwNS9Y0hLBgLqxf3cAAAAAAAEBK400lwAAAAAAIlEgC450hrwwagrvt6fACvBAVULbGs1z7syoJ3HM9f5etg8BAwQBAAAAIRYCkIHs5WFqocuZMZ/Eh07+5H8IzrpfYARjbIxDQJpfChkA9azC/VYAAIABAACAAAAAgAEAAAACAAAAARcgApCB7OVhaqHLmTGfxIdO/uR/CM66X2AEY2yMQ0CaXwoAAQErECcAAAAAAAAiUSBBRhSJpDy2fixk85QnJ47dMMA9bXKkuJmGFHxCjwzF8QEDBAEAAAAhFur3ZTRDvsZUN9uh2lL5ypfiMXha9WYn6PnDEc9zAPbeGQD1rML9VgAAgAEAAIAAAACAAAAAAAIAAAABFyDq92U0Q77GVDfbodpS+cqX4jF4WvVmJ+j5wxHPcwD23gABBSACkIHs5WFqocuZMZ/Eh07+5H8IzrpfYARjbIxDQJpfCiEHApCB7OVhaqHLmTGfxIdO/uR/CM66X2AEY2yMQ0CaXwoZAPWswv1WAACAAQAAgAAAAIABAAAAAgAAAAAA


In [4]:
# signature produced for the input:

input_sig0 = b'\xb8.i1\xaf\xeb\xb0\xa0{\xad\xfaU1\xc9f\\@\x1f_\x15\xe2\xdd\xf2\xf7\x052\x91\xc1PR\xd3\x83\xe9\xd8\xbe\xc2\xbbhKT;\x97\xaem\xb3>%\xa0\xa6\x04\xfdQy\x03,k}\x97\xc1\x12\x0b\xc6\x16e'
input_sig1 = b'\x0e\xf8\xa4hSZ\x16c\xfe\x9e{|Q\x1bz\xe3y\xec\xf4\xc5A\x86#7\xbed\xab\xa9r\xa7\x95?H\x08G\xe3Ut\\\x94d{w\x9a\x1b\xe6\xe8s\r\xcd\xbb\xe12mk\xd1O\xf9J\xe6\x89\xda\xb9\x11'
input_sig0 = b'\x99\xc2\xaa_oi)\xe5\xad\xfd\xc9x\xa8g\xfd\xae\x94"|\xbf\x8d\x0c\xaa\x1a\xe1\xb2{\x88\xfeJ\xc2\xbaa\xdaw\x95"k\xc3\x81Gj\xd6\xf1B\x028\x812^2\xd7%#\xf3\x15\x87\xb2\x80\'\xecG>G\x01'
input_sig1 = b'\xc5r\x19n\x9a\xa5\x0c\x9e\xb5\xa1\x99m\x12\xfc\x7fW\xe5\xff\x82f6\xe0iY\xfap!\xaa\xef\x8e\xb8\x8a\t\xa08\xc6\x03;j\xcbv X\x08\xc4\xd7\xa8\x8f\x1a\x15\xd4\x82N0\x1e\x194h<;\xc2\xc4|\x98\x01'
stx = CTransaction()
stx.deserialize(BytesIO(psbt.tx.serialize()))

in_wit0 = CTxInWitness()
in_wit0.scriptWitness.stack = [input_sig0]

in_wit1 = CTxInWitness()
in_wit1.scriptWitness.stack = [input_sig1]

wit = CTxWitness()
wit.vtxinwit = [in_wit0, in_wit1]

wit.serialize()

stx.wit = wit
stx.serialize_with_witness().hex()


#ACCEPTED BY MEMPOOL

'0200000000010216f79a3163ec37e83c54c6e2e8ff6cd8838e835ef3adc3dfd59627c949ec24090000000000fdfffffff9e21b877c81e1862254fd03941c0391122854284c6bb7fbb11d8acdcaeac0b30000000000fdffffff02d2339700000000002251200b8e7486bc306a0aefb7a7c00af0405542db1acd73eecca82771ccf5fe5eb60f55260000000000001600141399b8a09847a039b0352f58d212c180bab17f77014199c2aa5f6f6929e5adfdc978a867fdae94227cbf8d0caa1ae1b27b88fe4ac2ba61da7795226bc381476ad6f142023881325e32d72523f31587b28027ec473e47010141c572196e9aa50c9eb5a1996d12fc7f57e5ff826636e06959fa7021aaef8eb88a09a038c6033b6acb76205808c4d7a88f1a15d4824e301e1934683c3bc2c47c980100000000'

In [5]:
# SIGHASH_NONE_SIGN
psbt = PSBT()
psbt.deserialize(PSBT_IMPLICIT_ALL)

# input 0 
input_pk0 = bip32.get_pubkey_from_path("m/86'/1'/0'/1/2")
input_pk0 = input_pk0[1:] 

psbt.tx.vin[0].prevout.hash = int("0924ec49c92796d5dfc3adf35e838e83d86cffe8e2c6543ce837ec63319af716", 16)
psbt.tx.vin[0].prevout.n = 0
psbt.tx.vin[0].nSequence = 4294967293
psbt.tx.vout[0].nValue = 9909202
psbt.inputs[0].sighash = 2
psbt.inputs[0].witness_utxo.nValue = 9909389
psbt.inputs[0].witness_utxo.scriptPubKey = bytes.fromhex("51200b8e7486bc306a0aefb7a7c00af0405542db1acd73eecca82771ccf5fe5eb60f")
psbt.inputs[0].tap_internal_key=bytes.fromhex("029081ece5616aa1cb99319fc4874efee47f08ceba5f6004636c8c43409a5f0a")
psbt.inputs[0].tap_bip32_paths = {input_pk0: (set(), KeyOriginInfo(fpr, [86^H, 1^H, 0^H, 1, 2]))}

psbt.tx.vin.append(CTxIn())
psbt.inputs.append(PartiallySignedInput(0))

#input 1
psbt.inputs[1].witness_utxo = CTxOut()

input_pk1 = bip32.get_pubkey_from_path("m/86'/1'/0'/0/2")
input_pk1 = input_pk1[1:] 

psbt.tx.vin[1].prevout.hash = int("b3c0eacacd8a1db1fbb76b4c2854281291031c9403fd542286e1817c871be2f9", 16)
psbt.tx.vin[1].prevout.n = 0
psbt.tx.vin[1].nSequence = 4294967293
psbt.tx.vout[1].nValue = 9813
psbt.inputs[1].sighash = 2
psbt.inputs[1].witness_utxo.nValue = 10000
psbt.inputs[1].witness_utxo.scriptPubKey = bytes.fromhex("512041461489a43cb67e2c64f39427278edd30c03d6d72a4b89986147c428f0cc5f1")
psbt.inputs[1].tap_internal_key=bytes.fromhex("eaf7653443bec65437dba1da52f9ca97e231785af56627e8f9c311cf7300f6de")
psbt.inputs[1].tap_bip32_paths = {input_pk1: (set(), KeyOriginInfo(fpr, [86^H, 1^H, 0^H, 0, 2]))}

psbt.inputs[1].witness_utxo.scriptPubKey.hex()

NEW_PSBT = psbt.serialize()
print(NEW_PSBT)

cHNidP8BAKYCAAAAAhb3mjFj7DfoPFTG4uj/bNiDjoNe863D39WWJ8lJ7CQJAAAAAAD9////+eIbh3yB4YYiVP0DlBwDkRIoVChMa7f7sR2KzcrqwLMAAAAAAP3///8C0jOXAAAAAAAiUSALjnSGvDBqCu+3p8AK8EBVQtsazXPuzKgnccz1/l62D1UmAAAAAAAAFgAUE5m4oJhHoDmwNS9Y0hLBgLqxf3cAAAAAAAEBK400lwAAAAAAIlEgC450hrwwagrvt6fACvBAVULbGs1z7syoJ3HM9f5etg8BAwQCAAAAIRYCkIHs5WFqocuZMZ/Eh07+5H8IzrpfYARjbIxDQJpfChkA9azC/VYAAIABAACAAAAAgAEAAAACAAAAARcgApCB7OVhaqHLmTGfxIdO/uR/CM66X2AEY2yMQ0CaXwoAAQErECcAAAAAAAAiUSBBRhSJpDy2fixk85QnJ47dMMA9bXKkuJmGFHxCjwzF8QEDBAIAAAAhFur3ZTRDvsZUN9uh2lL5ypfiMXha9WYn6PnDEc9zAPbeGQD1rML9VgAAgAEAAIAAAACAAAAAAAIAAAABFyDq92U0Q77GVDfbodpS+cqX4jF4WvVmJ+j5wxHPcwD23gABBSACkIHs5WFqocuZMZ/Eh07+5H8IzrpfYARjbIxDQJpfCiEHApCB7OVhaqHLmTGfxIdO/uR/CM66X2AEY2yMQ0CaXwoZAPWswv1WAACAAQAAgAAAAIABAAAAAgAAAAAA


In [6]:
# signature produced for the input:
input_sig0 = b'm\x8a\x907&\xf38\xa8\xa0\xa8\x18y\xc9\xaam\xa5\x056O\xf3\x08\x1aq-\xad\x96q\x04m\xc2\x0em>\xe3E\x06\xab"6\x1c\xe0\xd6\xb98N\x1e\x18\xaa\xb3Hr\xdc\xbexI\x9aQa7+\x02Z\xe5)\x02'
input_sig1 = b'\xf6\\\xdc,\x8fIl\xa9\x98\x19\x05\x90\xac\x0e\xdc\xe1K>\xa4\xaeGy\xb4\x05z\x0eh\x9f*G\x9e\x90S\xc0i\xba\xf6\xd6\xb9w\xb3\x02\xbd\xd5\xd7\xa6W\xe0\xde\xb0Q\xeep\xce\xa2BS\xd0t\x19\x1f\xb2O\xc5\x02'

stx = CTransaction()
stx.deserialize(BytesIO(psbt.tx.serialize()))

in_wit0 = CTxInWitness()
in_wit0.scriptWitness.stack = [input_sig0]

in_wit1 = CTxInWitness()
in_wit1.scriptWitness.stack = [input_sig1]

wit = CTxWitness()
wit.vtxinwit = [in_wit0, in_wit1]

wit.serialize()

stx.wit = wit
stx.serialize_with_witness().hex()

#ACCEPTED BY MEMPOOL

'0200000000010216f79a3163ec37e83c54c6e2e8ff6cd8838e835ef3adc3dfd59627c949ec24090000000000fdfffffff9e21b877c81e1862254fd03941c0391122854284c6bb7fbb11d8acdcaeac0b30000000000fdffffff02d2339700000000002251200b8e7486bc306a0aefb7a7c00af0405542db1acd73eecca82771ccf5fe5eb60f55260000000000001600141399b8a09847a039b0352f58d212c180bab17f7701416d8a903726f338a8a0a81879c9aa6da505364ff3081a712dad9671046dc20e6d3ee34506ab22361ce0d6b9384e1e18aab34872dcbe78499a5161372b025ae529020141f65cdc2c8f496ca998190590ac0edce14b3ea4ae4779b4057a0e689f2a479e9053c069baf6d6b977b302bdd5d7a657e0deb051ee70cea24253d074191fb24fc50200000000'

In [7]:
# SIGHASH_NONE_1_IN_2_OUTS
psbt = PSBT()
psbt.deserialize(PSBT_IMPLICIT_ALL)

# input 0 
input_pk0 = bip32.get_pubkey_from_path("m/86'/1'/0'/1/2")
input_pk0 = input_pk0[1:] 

psbt.tx.vin[0].prevout.hash = int("0924ec49c92796d5dfc3adf35e838e83d86cffe8e2c6543ce837ec63319af716", 16)
psbt.tx.vin[0].prevout.n = 0
psbt.tx.vin[0].nSequence = 4294967293
psbt.tx.vout[0].nValue = 9909202
psbt.inputs[0].sighash = 2
psbt.inputs[0].witness_utxo.nValue = 9909389
psbt.inputs[0].witness_utxo.scriptPubKey = bytes.fromhex("51200b8e7486bc306a0aefb7a7c00af0405542db1acd73eecca82771ccf5fe5eb60f")
psbt.inputs[0].tap_internal_key=bytes.fromhex("029081ece5616aa1cb99319fc4874efee47f08ceba5f6004636c8c43409a5f0a")
psbt.inputs[0].tap_bip32_paths = {input_pk0: (set(), KeyOriginInfo(fpr, [86^H, 1^H, 0^H, 1, 2]))}


NEW_PSBT = psbt.serialize()
print(NEW_PSBT)

cHNidP8BAH0CAAAAARb3mjFj7DfoPFTG4uj/bNiDjoNe863D39WWJ8lJ7CQJAAAAAAD9////AtIzlwAAAAAAIlEgC450hrwwagrvt6fACvBAVULbGs1z7syoJ3HM9f5etg+ghgEAAAAAABYAFBOZuKCYR6A5sDUvWNISwYC6sX93AAAAAAABASuNNJcAAAAAACJRIAuOdIa8MGoK77enwArwQFVC2xrNc+7MqCdxzPX+XrYPAQMEAgAAACEWApCB7OVhaqHLmTGfxIdO/uR/CM66X2AEY2yMQ0CaXwoZAPWswv1WAACAAQAAgAAAAIABAAAAAgAAAAEXIAKQgezlYWqhy5kxn8SHTv7kfwjOul9gBGNsjENAml8KAAEFIAKQgezlYWqhy5kxn8SHTv7kfwjOul9gBGNsjENAml8KIQcCkIHs5WFqocuZMZ/Eh07+5H8IzrpfYARjbIxDQJpfChkA9azC/VYAAIABAACAAAAAgAEAAAACAAAAAAA=


In [8]:
# SIGHASH_SINGLE_SIGN
psbt = PSBT()
psbt.deserialize(PSBT_IMPLICIT_ALL)

# input 0 
input_pk0 = bip32.get_pubkey_from_path("m/86'/1'/0'/1/2")
input_pk0 = input_pk0[1:] 

psbt.tx.vin[0].prevout.hash = int("0924ec49c92796d5dfc3adf35e838e83d86cffe8e2c6543ce837ec63319af716", 16)
psbt.tx.vin[0].prevout.n = 0
psbt.tx.vin[0].nSequence = 4294967293
psbt.tx.vout[0].nValue = 9909202
psbt.inputs[0].sighash = 3
psbt.inputs[0].witness_utxo.nValue = 9909389
psbt.inputs[0].witness_utxo.scriptPubKey = bytes.fromhex("51200b8e7486bc306a0aefb7a7c00af0405542db1acd73eecca82771ccf5fe5eb60f")
psbt.inputs[0].tap_internal_key=bytes.fromhex("029081ece5616aa1cb99319fc4874efee47f08ceba5f6004636c8c43409a5f0a")
psbt.inputs[0].tap_bip32_paths = {input_pk0: (set(), KeyOriginInfo(fpr, [86^H, 1^H, 0^H, 1, 2]))}

psbt.tx.vin.append(CTxIn())
psbt.inputs.append(PartiallySignedInput(0))

#input 1
psbt.inputs[1].witness_utxo = CTxOut()

input_pk1 = bip32.get_pubkey_from_path("m/86'/1'/0'/0/2")
input_pk1 = input_pk1[1:] 

psbt.tx.vin[1].prevout.hash = int("b3c0eacacd8a1db1fbb76b4c2854281291031c9403fd542286e1817c871be2f9", 16)
psbt.tx.vin[1].prevout.n = 0
psbt.tx.vin[1].nSequence = 4294967293
psbt.tx.vout[1].nValue = 9813
psbt.inputs[1].sighash = 3
psbt.inputs[1].witness_utxo.nValue = 10000
psbt.inputs[1].witness_utxo.scriptPubKey = bytes.fromhex("512041461489a43cb67e2c64f39427278edd30c03d6d72a4b89986147c428f0cc5f1")
psbt.inputs[1].tap_internal_key=bytes.fromhex("eaf7653443bec65437dba1da52f9ca97e231785af56627e8f9c311cf7300f6de")
psbt.inputs[1].tap_bip32_paths = {input_pk1: (set(), KeyOriginInfo(fpr, [86^H, 1^H, 0^H, 0, 2]))}

psbt.inputs[1].witness_utxo.scriptPubKey.hex()

NEW_PSBT = psbt.serialize()

print(NEW_PSBT)

cHNidP8BAKYCAAAAAhb3mjFj7DfoPFTG4uj/bNiDjoNe863D39WWJ8lJ7CQJAAAAAAD9////+eIbh3yB4YYiVP0DlBwDkRIoVChMa7f7sR2KzcrqwLMAAAAAAP3///8C0jOXAAAAAAAiUSALjnSGvDBqCu+3p8AK8EBVQtsazXPuzKgnccz1/l62D1UmAAAAAAAAFgAUE5m4oJhHoDmwNS9Y0hLBgLqxf3cAAAAAAAEBK400lwAAAAAAIlEgC450hrwwagrvt6fACvBAVULbGs1z7syoJ3HM9f5etg8BAwQDAAAAIRYCkIHs5WFqocuZMZ/Eh07+5H8IzrpfYARjbIxDQJpfChkA9azC/VYAAIABAACAAAAAgAEAAAACAAAAARcgApCB7OVhaqHLmTGfxIdO/uR/CM66X2AEY2yMQ0CaXwoAAQErECcAAAAAAAAiUSBBRhSJpDy2fixk85QnJ47dMMA9bXKkuJmGFHxCjwzF8QEDBAMAAAAhFur3ZTRDvsZUN9uh2lL5ypfiMXha9WYn6PnDEc9zAPbeGQD1rML9VgAAgAEAAIAAAACAAAAAAAIAAAABFyDq92U0Q77GVDfbodpS+cqX4jF4WvVmJ+j5wxHPcwD23gABBSACkIHs5WFqocuZMZ/Eh07+5H8IzrpfYARjbIxDQJpfCiEHApCB7OVhaqHLmTGfxIdO/uR/CM66X2AEY2yMQ0CaXwoZAPWswv1WAACAAQAAgAAAAIABAAAAAgAAAAAA


In [9]:
# signature produced for the input:
input_sig0 =b'\xde\xed?\xd3\x1dh/\xcaat\x91\xdf\x15\x1fQ\x0f\xd0\xc5\x1b}\xda_\xf4\xaf\x86\xfb\xfc\x91fZ\xd1\x03h\xa9\xde, \xdd\x8a\xdc-bni8M\xdfC\xe6SE\xdf(\xbe\x81\x1dmX\xfc$\x85\xd5\xd0\xff\x03'
input_sig1 =b'{\xe5)\x97\xe1\xfd*\xc3\xde\xa2\xcf\x9ai8\xc4\x13\xef\xc8\xf7\xf9\xfc\xfd\xd2\x9d\x0c,\x13\xd4*Y\xd2\x97!\xf4eq4zco\xf8\x96\xc0\xbe\xaf\xa7\xbdhY\x0c\xc9O\xd7\x9cL\xa9\xe0~\xef\x02\x97\xbf.\x91\x03'

stx = CTransaction()
stx.deserialize(BytesIO(psbt.tx.serialize()))

in_wit0 = CTxInWitness()
in_wit0.scriptWitness.stack = [input_sig0]

in_wit1 = CTxInWitness()
in_wit1.scriptWitness.stack = [input_sig1]

wit = CTxWitness()
wit.vtxinwit = [in_wit0, in_wit1]

wit.serialize()

stx.wit = wit
stx.serialize_with_witness().hex()

#ACCEPTED BY MEMPOOL

'0200000000010216f79a3163ec37e83c54c6e2e8ff6cd8838e835ef3adc3dfd59627c949ec24090000000000fdfffffff9e21b877c81e1862254fd03941c0391122854284c6bb7fbb11d8acdcaeac0b30000000000fdffffff02d2339700000000002251200b8e7486bc306a0aefb7a7c00af0405542db1acd73eecca82771ccf5fe5eb60f55260000000000001600141399b8a09847a039b0352f58d212c180bab17f770141deed3fd31d682fca617491df151f510fd0c51b7dda5ff4af86fbfc91665ad10368a9de2c20dd8adc2d626e69384ddf43e65345df28be811d6d58fc2485d5d0ff0301417be52997e1fd2ac3dea2cf9a6938c413efc8f7f9fcfdd29d0c2c13d42a59d29721f46571347a636ff896c0beafa7bd68590cc94fd79c4ca9e07eef0297bf2e910300000000'

In [10]:
# SIGHASH_SINGLE_3_INS_2_OUT
psbt = PSBT()
psbt.deserialize(PSBT_IMPLICIT_ALL)

# input 0 
input_pk0 = bip32.get_pubkey_from_path("m/86'/1'/0'/1/2")
input_pk0 = input_pk0[1:] 

psbt.tx.vin[0].prevout.hash = int("0924ec49c92796d5dfc3adf35e838e83d86cffe8e2c6543ce837ec63319af716", 16)
psbt.tx.vin[0].prevout.n = 0
psbt.tx.vin[0].nSequence = 4294967293
psbt.tx.vout[0].nValue = 9909202
psbt.inputs[0].sighash = 3
psbt.inputs[0].witness_utxo.nValue = 9909389
psbt.inputs[0].witness_utxo.scriptPubKey = bytes.fromhex("51200b8e7486bc306a0aefb7a7c00af0405542db1acd73eecca82771ccf5fe5eb60f")
psbt.inputs[0].tap_internal_key=bytes.fromhex("029081ece5616aa1cb99319fc4874efee47f08ceba5f6004636c8c43409a5f0a")
psbt.inputs[0].tap_bip32_paths = {input_pk0: (set(), KeyOriginInfo(fpr, [86^H, 1^H, 0^H, 1, 2]))}

psbt.tx.vin.append(CTxIn())
psbt.inputs.append(PartiallySignedInput(0))

#input 1
psbt.inputs[1].witness_utxo = CTxOut()

input_pk1 = bip32.get_pubkey_from_path("m/86'/1'/0'/0/2")
input_pk1 = input_pk1[1:] 

psbt.tx.vin[1].prevout.hash = int("b3c0eacacd8a1db1fbb76b4c2854281291031c9403fd542286e1817c871be2f9", 16)
psbt.tx.vin[1].prevout.n = 0
psbt.tx.vin[1].nSequence = 4294967293
psbt.tx.vout[1].nValue = 9813
psbt.inputs[1].sighash = 3
psbt.inputs[1].witness_utxo.nValue = 10000
psbt.inputs[1].witness_utxo.scriptPubKey = bytes.fromhex("512041461489a43cb67e2c64f39427278edd30c03d6d72a4b89986147c428f0cc5f1")
psbt.inputs[1].tap_internal_key=bytes.fromhex("eaf7653443bec65437dba1da52f9ca97e231785af56627e8f9c311cf7300f6de")
psbt.inputs[1].tap_bip32_paths = {input_pk1: (set(), KeyOriginInfo(fpr, [86^H, 1^H, 0^H, 0, 2]))}

psbt.inputs[1].witness_utxo.scriptPubKey.hex()
psbt.tx.vin.append(CTxIn())
psbt.inputs.append(PartiallySignedInput(0))

#input 2
psbt.inputs[2].witness_utxo = CTxOut()

input_pk2 = bip32.get_pubkey_from_path("m/86'/1'/0'/0/4")
input_pk2 = input_pk2[1:] 

psbt.tx.vin[2].prevout.hash = int("b4d12567b88347439690a1220a89c53b2cf6409c3b597ca2f6e323486fde5973", 16)
psbt.tx.vin[2].prevout.n = 0
psbt.tx.vin[2].nSequence = 4294967293
psbt.inputs[2].sighash = 3
psbt.inputs[2].witness_utxo.nValue = 100000
psbt.inputs[2].witness_utxo.scriptPubKey = bytes.fromhex("51206849909c22d925c2abb695c1406216bed2fd6847524dc64374f6beb2bf15987f")
psbt.inputs[2].tap_internal_key=bytes.fromhex("6a39916f5bcbb06e2f3fe25a4d9b12b4092fc81a7b67cd25de46e0ba10a53607")
psbt.inputs[2].tap_bip32_paths = {input_pk2: (set(), KeyOriginInfo(fpr, [86^H, 1^H, 0^H, 0, 4]))}

psbt.inputs[2].witness_utxo.scriptPubKey.hex()


NEW_PSBT = psbt.serialize()
print(NEW_PSBT)

cHNidP8BAM8CAAAAAxb3mjFj7DfoPFTG4uj/bNiDjoNe863D39WWJ8lJ7CQJAAAAAAD9////+eIbh3yB4YYiVP0DlBwDkRIoVChMa7f7sR2KzcrqwLMAAAAAAP3///9zWd5vSCPj9qJ8WTucQPYsO8WJCiKhkJZDR4O4ZyXRtAAAAAAA/f///wLSM5cAAAAAACJRIAuOdIa8MGoK77enwArwQFVC2xrNc+7MqCdxzPX+XrYPVSYAAAAAAAAWABQTmbigmEegObA1L1jSEsGAurF/dwAAAAAAAQErjTSXAAAAAAAiUSALjnSGvDBqCu+3p8AK8EBVQtsazXPuzKgnccz1/l62DwEDBAMAAAAhFgKQgezlYWqhy5kxn8SHTv7kfwjOul9gBGNsjENAml8KGQD1rML9VgAAgAEAAIAAAACAAQAAAAIAAAABFyACkIHs5WFqocuZMZ/Eh07+5H8IzrpfYARjbIxDQJpfCgABASsQJwAAAAAAACJRIEFGFImkPLZ+LGTzlCcnjt0wwD1tcqS4mYYUfEKPDMXxAQMEAwAAACEW6vdlNEO+xlQ326HaUvnKl+IxeFr1Zifo+cMRz3MA9t4ZAPWswv1WAACAAQAAgAAAAIAAAAAAAgAAAAEXIOr3ZTRDvsZUN9uh2lL5ypfiMXha9WYn6PnDEc9zAPbeAAEBK6CGAQAAAAAAIlEgaEmQnCLZJcKrtpXBQGIWvtL9aEdSTcZDdPa+sr8VmH8BAwQDAAAAIRZqOZFvW8uwbi8/4lpNmxK0CS/IGntnzSXeRuC6EKU2BxkA9azC/VYAAIABAACAAAAAgAAAAAAEAAAAARcgajmRb1vLsG4vP+JaTZsStAkvyBp7Z80l3kbguhClNgcAAQUgApCB7OVhaqHLmTGfxIdO/uR/CM66X2AEY2yMQ0CaXwohBwKQgezlYWqhy5kxn8SHTv7kfwjOul9gBGNsjENAml8KGQD1rML9VgAAgAEAAIAAAACA

In [11]:
# SIGHASH_ALL_ANYONE_SIGN
psbt = PSBT()
psbt.deserialize(PSBT_IMPLICIT_ALL)

# input 0 
input_pk0 = bip32.get_pubkey_from_path("m/86'/1'/0'/1/2")
input_pk0 = input_pk0[1:] 

psbt.tx.vin[0].prevout.hash = int("0924ec49c92796d5dfc3adf35e838e83d86cffe8e2c6543ce837ec63319af716", 16)
psbt.tx.vin[0].prevout.n = 0
psbt.tx.vin[0].nSequence = 4294967293
psbt.tx.vout[0].nValue = 9909202
psbt.inputs[0].sighash = 0x81
psbt.inputs[0].witness_utxo.nValue = 9909389
psbt.inputs[0].witness_utxo.scriptPubKey = bytes.fromhex("51200b8e7486bc306a0aefb7a7c00af0405542db1acd73eecca82771ccf5fe5eb60f")
psbt.inputs[0].tap_internal_key=bytes.fromhex("029081ece5616aa1cb99319fc4874efee47f08ceba5f6004636c8c43409a5f0a")
psbt.inputs[0].tap_bip32_paths = {input_pk0: (set(), KeyOriginInfo(fpr, [86^H, 1^H, 0^H, 1, 2]))}

psbt.tx.vin.append(CTxIn())
psbt.inputs.append(PartiallySignedInput(0))

#input 1
psbt.inputs[1].witness_utxo = CTxOut()

input_pk1 = bip32.get_pubkey_from_path("m/86'/1'/0'/0/2")
input_pk1 = input_pk1[1:] 

psbt.tx.vin[1].prevout.hash = int("b3c0eacacd8a1db1fbb76b4c2854281291031c9403fd542286e1817c871be2f9", 16)
psbt.tx.vin[1].prevout.n = 0
psbt.tx.vin[1].nSequence = 4294967293
psbt.tx.vout[1].nValue = 9813
psbt.inputs[1].sighash = 0x81
psbt.inputs[1].witness_utxo.nValue = 10000
psbt.inputs[1].witness_utxo.scriptPubKey = bytes.fromhex("512041461489a43cb67e2c64f39427278edd30c03d6d72a4b89986147c428f0cc5f1")
psbt.inputs[1].tap_internal_key=bytes.fromhex("eaf7653443bec65437dba1da52f9ca97e231785af56627e8f9c311cf7300f6de")
psbt.inputs[1].tap_bip32_paths = {input_pk1: (set(), KeyOriginInfo(fpr, [86^H, 1^H, 0^H, 0, 2]))}

psbt.inputs[1].witness_utxo.scriptPubKey.hex()

NEW_PSBT = psbt.serialize()
print(NEW_PSBT)

cHNidP8BAKYCAAAAAhb3mjFj7DfoPFTG4uj/bNiDjoNe863D39WWJ8lJ7CQJAAAAAAD9////+eIbh3yB4YYiVP0DlBwDkRIoVChMa7f7sR2KzcrqwLMAAAAAAP3///8C0jOXAAAAAAAiUSALjnSGvDBqCu+3p8AK8EBVQtsazXPuzKgnccz1/l62D1UmAAAAAAAAFgAUE5m4oJhHoDmwNS9Y0hLBgLqxf3cAAAAAAAEBK400lwAAAAAAIlEgC450hrwwagrvt6fACvBAVULbGs1z7syoJ3HM9f5etg8BAwSBAAAAIRYCkIHs5WFqocuZMZ/Eh07+5H8IzrpfYARjbIxDQJpfChkA9azC/VYAAIABAACAAAAAgAEAAAACAAAAARcgApCB7OVhaqHLmTGfxIdO/uR/CM66X2AEY2yMQ0CaXwoAAQErECcAAAAAAAAiUSBBRhSJpDy2fixk85QnJ47dMMA9bXKkuJmGFHxCjwzF8QEDBIEAAAAhFur3ZTRDvsZUN9uh2lL5ypfiMXha9WYn6PnDEc9zAPbeGQD1rML9VgAAgAEAAIAAAACAAAAAAAIAAAABFyDq92U0Q77GVDfbodpS+cqX4jF4WvVmJ+j5wxHPcwD23gABBSACkIHs5WFqocuZMZ/Eh07+5H8IzrpfYARjbIxDQJpfCiEHApCB7OVhaqHLmTGfxIdO/uR/CM66X2AEY2yMQ0CaXwoZAPWswv1WAACAAQAAgAAAAIABAAAAAgAAAAAA


In [12]:
# signature produced for the input:

# signature produced for the input:
input_sig0 = b'\xac\xe4:\x06\xb1\x843\xd8\x87\xce\x9e\x15\x9c\xe6m\xdf\xe4\xa6\xf4\xcf\x888\xb3UER\xcc\xaf\x9dRGE\xfeylJ\x0e3\xf5\xe9\x9e\xcf\xdby\xbf\x11\xb8V\x95\x8d\xd2*\x8f\xe99\x88\xb0\xfb!\x90\x83\xd4\xb5g\x81'
input_sig1 = b'\xd2\x00}\xadR\x80\x1c4\xd6\xf4Z\x976u\x82\x19+\x04xe_\xcfQ\xa8\xb3\xc5\xd8W\xc6\xce\x92\x06\xb4)x2\xcb\xf1\xf0\x074C<\xa4W1\xa5\x84\xdbx\xfes\xbc\xae\xcc\t\xcfs\xec\x03B\xd1\xdd\xe2\x81'

stx = CTransaction()
stx.deserialize(BytesIO(psbt.tx.serialize()))

in_wit0 = CTxInWitness()
in_wit0.scriptWitness.stack = [input_sig0]

in_wit1 = CTxInWitness()
in_wit1.scriptWitness.stack = [input_sig1]

wit = CTxWitness()
wit.vtxinwit = [in_wit0, in_wit1]

wit.serialize()

stx.wit = wit
stx.serialize_with_witness().hex()

#ACCEPTED BY MEMPOOL

'0200000000010216f79a3163ec37e83c54c6e2e8ff6cd8838e835ef3adc3dfd59627c949ec24090000000000fdfffffff9e21b877c81e1862254fd03941c0391122854284c6bb7fbb11d8acdcaeac0b30000000000fdffffff02d2339700000000002251200b8e7486bc306a0aefb7a7c00af0405542db1acd73eecca82771ccf5fe5eb60f55260000000000001600141399b8a09847a039b0352f58d212c180bab17f770141ace43a06b18433d887ce9e159ce66ddfe4a6f4cf8838b3554552ccaf9d524745fe796c4a0e33f5e99ecfdb79bf11b856958dd22a8fe93988b0fb219083d4b567810141d2007dad52801c34d6f45a97367582192b0478655fcf51a8b3c5d857c6ce9206b4297832cbf1f00734433ca45731a584db78fe73bcaecc09cf73ec0342d1dde28100000000'

In [13]:
# SIGHASH_NONE_ANYONE_SIGN
psbt = PSBT()
psbt.deserialize(PSBT_IMPLICIT_ALL)

# input 0 
input_pk0 = bip32.get_pubkey_from_path("m/86'/1'/0'/1/2")
input_pk0 = input_pk0[1:] 

psbt.tx.vin[0].prevout.hash = int("0924ec49c92796d5dfc3adf35e838e83d86cffe8e2c6543ce837ec63319af716", 16)
psbt.tx.vin[0].prevout.n = 0
psbt.tx.vin[0].nSequence = 4294967293
psbt.tx.vout[0].nValue = 9909202
psbt.inputs[0].sighash = 0x82
psbt.inputs[0].witness_utxo.nValue = 9909389
psbt.inputs[0].witness_utxo.scriptPubKey = bytes.fromhex("51200b8e7486bc306a0aefb7a7c00af0405542db1acd73eecca82771ccf5fe5eb60f")
psbt.inputs[0].tap_internal_key=bytes.fromhex("029081ece5616aa1cb99319fc4874efee47f08ceba5f6004636c8c43409a5f0a")
psbt.inputs[0].tap_bip32_paths = {input_pk0: (set(), KeyOriginInfo(fpr, [86^H, 1^H, 0^H, 1, 2]))}

psbt.tx.vin.append(CTxIn())
psbt.inputs.append(PartiallySignedInput(0))

#input 1
psbt.inputs[1].witness_utxo = CTxOut()

input_pk1 = bip32.get_pubkey_from_path("m/86'/1'/0'/0/2")
input_pk1 = input_pk1[1:] 

psbt.tx.vin[1].prevout.hash = int("b3c0eacacd8a1db1fbb76b4c2854281291031c9403fd542286e1817c871be2f9", 16)
psbt.tx.vin[1].prevout.n = 0
psbt.tx.vin[1].nSequence = 4294967293
psbt.tx.vout[1].nValue = 9813
psbt.inputs[1].sighash = 0x82
psbt.inputs[1].witness_utxo.nValue = 10000
psbt.inputs[1].witness_utxo.scriptPubKey = bytes.fromhex("512041461489a43cb67e2c64f39427278edd30c03d6d72a4b89986147c428f0cc5f1")
psbt.inputs[1].tap_internal_key=bytes.fromhex("eaf7653443bec65437dba1da52f9ca97e231785af56627e8f9c311cf7300f6de")
psbt.inputs[1].tap_bip32_paths = {input_pk1: (set(), KeyOriginInfo(fpr, [86^H, 1^H, 0^H, 0, 2]))}

psbt.inputs[1].witness_utxo.scriptPubKey.hex()

NEW_PSBT = psbt.serialize()
print(NEW_PSBT)

cHNidP8BAKYCAAAAAhb3mjFj7DfoPFTG4uj/bNiDjoNe863D39WWJ8lJ7CQJAAAAAAD9////+eIbh3yB4YYiVP0DlBwDkRIoVChMa7f7sR2KzcrqwLMAAAAAAP3///8C0jOXAAAAAAAiUSALjnSGvDBqCu+3p8AK8EBVQtsazXPuzKgnccz1/l62D1UmAAAAAAAAFgAUE5m4oJhHoDmwNS9Y0hLBgLqxf3cAAAAAAAEBK400lwAAAAAAIlEgC450hrwwagrvt6fACvBAVULbGs1z7syoJ3HM9f5etg8BAwSCAAAAIRYCkIHs5WFqocuZMZ/Eh07+5H8IzrpfYARjbIxDQJpfChkA9azC/VYAAIABAACAAAAAgAEAAAACAAAAARcgApCB7OVhaqHLmTGfxIdO/uR/CM66X2AEY2yMQ0CaXwoAAQErECcAAAAAAAAiUSBBRhSJpDy2fixk85QnJ47dMMA9bXKkuJmGFHxCjwzF8QEDBIIAAAAhFur3ZTRDvsZUN9uh2lL5ypfiMXha9WYn6PnDEc9zAPbeGQD1rML9VgAAgAEAAIAAAACAAAAAAAIAAAABFyDq92U0Q77GVDfbodpS+cqX4jF4WvVmJ+j5wxHPcwD23gABBSACkIHs5WFqocuZMZ/Eh07+5H8IzrpfYARjbIxDQJpfCiEHApCB7OVhaqHLmTGfxIdO/uR/CM66X2AEY2yMQ0CaXwoZAPWswv1WAACAAQAAgAAAAIABAAAAAgAAAAAA


In [14]:
# signature produced for the input:
input_sig0 =b'\xf0\x8e\xadW\xbdK\xc0\xe0\xbc\x17\x9cd\xc9Z.W#\xb0?\xb8\x8ccm\x9dq5\x01\xe8\xdd*K\xf4\xf7mr\xee}^T\xa8y\xa8\xd6\xbb\x98\xd6v$b\xeaj\xf5pe\x14\x87\\4s^\xea5]\x10\x82'
input_sig1 = b"\xfb\x849\xd9r@\xa9\x92\r#nM\x14R\x08I\x1c\xa9\xf0\x80^3c\x85\x1c\xc4\xcc\xb2'm3Z\xa8\x9a\xbf^\xea\x85#\xd7\xab\x99\x1d#\xab\x19y\x0cI^\xc2\xa4c\x13\xc9\xc2\x98\xb8`\x9eM\xf8\xbaX\x82"
stx = CTransaction()
stx.deserialize(BytesIO(psbt.tx.serialize()))

in_wit0 = CTxInWitness()
in_wit0.scriptWitness.stack = [input_sig0]

in_wit1 = CTxInWitness()
in_wit1.scriptWitness.stack = [input_sig1]

wit = CTxWitness()
wit.vtxinwit = [in_wit0, in_wit1]

wit.serialize()

stx.wit = wit
stx.serialize_with_witness().hex()
#ACCEPTED BY MEMPOOL

'0200000000010216f79a3163ec37e83c54c6e2e8ff6cd8838e835ef3adc3dfd59627c949ec24090000000000fdfffffff9e21b877c81e1862254fd03941c0391122854284c6bb7fbb11d8acdcaeac0b30000000000fdffffff02d2339700000000002251200b8e7486bc306a0aefb7a7c00af0405542db1acd73eecca82771ccf5fe5eb60f55260000000000001600141399b8a09847a039b0352f58d212c180bab17f770141f08ead57bd4bc0e0bc179c64c95a2e5723b03fb88c636d9d713501e8dd2a4bf4f76d72ee7d5e54a879a8d6bb98d6762462ea6af5706514875c34735eea355d10820141fb8439d97240a9920d236e4d145208491ca9f0805e3363851cc4ccb2276d335aa89abf5eea8523d7ab991d23ab19790c495ec2a46313c9c298b8609e4df8ba588200000000'

In [15]:
# SIGHASH_SINGLE_ANYONE_SIGN
psbt = PSBT()
psbt.deserialize(PSBT_IMPLICIT_ALL)

# input 0 
input_pk0 = bip32.get_pubkey_from_path("m/86'/1'/0'/1/2")
input_pk0 = input_pk0[1:] 

psbt.tx.vin[0].prevout.hash = int("0924ec49c92796d5dfc3adf35e838e83d86cffe8e2c6543ce837ec63319af716", 16)
psbt.tx.vin[0].prevout.n = 0
psbt.tx.vin[0].nSequence = 4294967293
psbt.tx.vout[0].nValue = 9909202
psbt.inputs[0].sighash = 0x83
psbt.inputs[0].witness_utxo.nValue = 9909389
psbt.inputs[0].witness_utxo.scriptPubKey = bytes.fromhex("51200b8e7486bc306a0aefb7a7c00af0405542db1acd73eecca82771ccf5fe5eb60f")
psbt.inputs[0].tap_internal_key=bytes.fromhex("029081ece5616aa1cb99319fc4874efee47f08ceba5f6004636c8c43409a5f0a")
psbt.inputs[0].tap_bip32_paths = {input_pk0: (set(), KeyOriginInfo(fpr, [86^H, 1^H, 0^H, 1, 2]))}

psbt.tx.vin.append(CTxIn())
psbt.inputs.append(PartiallySignedInput(0))

#input 1
psbt.inputs[1].witness_utxo = CTxOut()

input_pk1 = bip32.get_pubkey_from_path("m/86'/1'/0'/0/2")
input_pk1 = input_pk1[1:] 

psbt.tx.vin[1].prevout.hash = int("b3c0eacacd8a1db1fbb76b4c2854281291031c9403fd542286e1817c871be2f9", 16)
psbt.tx.vin[1].prevout.n = 0
psbt.tx.vin[1].nSequence = 4294967293
psbt.tx.vout[1].nValue = 9813
psbt.inputs[1].sighash = 0x83
psbt.inputs[1].witness_utxo.nValue = 10000
psbt.inputs[1].witness_utxo.scriptPubKey = bytes.fromhex("512041461489a43cb67e2c64f39427278edd30c03d6d72a4b89986147c428f0cc5f1")
psbt.inputs[1].tap_internal_key=bytes.fromhex("eaf7653443bec65437dba1da52f9ca97e231785af56627e8f9c311cf7300f6de")
psbt.inputs[1].tap_bip32_paths = {input_pk1: (set(), KeyOriginInfo(fpr, [86^H, 1^H, 0^H, 0, 2]))}

psbt.inputs[1].witness_utxo.scriptPubKey.hex()

NEW_PSBT = psbt.serialize()
print(NEW_PSBT)

cHNidP8BAKYCAAAAAhb3mjFj7DfoPFTG4uj/bNiDjoNe863D39WWJ8lJ7CQJAAAAAAD9////+eIbh3yB4YYiVP0DlBwDkRIoVChMa7f7sR2KzcrqwLMAAAAAAP3///8C0jOXAAAAAAAiUSALjnSGvDBqCu+3p8AK8EBVQtsazXPuzKgnccz1/l62D1UmAAAAAAAAFgAUE5m4oJhHoDmwNS9Y0hLBgLqxf3cAAAAAAAEBK400lwAAAAAAIlEgC450hrwwagrvt6fACvBAVULbGs1z7syoJ3HM9f5etg8BAwSDAAAAIRYCkIHs5WFqocuZMZ/Eh07+5H8IzrpfYARjbIxDQJpfChkA9azC/VYAAIABAACAAAAAgAEAAAACAAAAARcgApCB7OVhaqHLmTGfxIdO/uR/CM66X2AEY2yMQ0CaXwoAAQErECcAAAAAAAAiUSBBRhSJpDy2fixk85QnJ47dMMA9bXKkuJmGFHxCjwzF8QEDBIMAAAAhFur3ZTRDvsZUN9uh2lL5ypfiMXha9WYn6PnDEc9zAPbeGQD1rML9VgAAgAEAAIAAAACAAAAAAAIAAAABFyDq92U0Q77GVDfbodpS+cqX4jF4WvVmJ+j5wxHPcwD23gABBSACkIHs5WFqocuZMZ/Eh07+5H8IzrpfYARjbIxDQJpfCiEHApCB7OVhaqHLmTGfxIdO/uR/CM66X2AEY2yMQ0CaXwoZAPWswv1WAACAAQAAgAAAAIABAAAAAgAAAAAA


In [16]:
# signature produced for the input:
input_sig0 = b'\x17Lg\xa5b\x9a\x05\xca\x88[\xb2^\xbcug>\xf1:9(\x8c\xce\xc7\x145\xd2-\x059\xd0z\xaf\x7f\xe5\xd5\xd9\xfbc\xeewh\xa5.}L8\xb4\x11m5\xd2\x9a\xf0\xe9\x8cH`Q\xa0\xcfboC8\x83'
input_sig1 = b'*\x90\xc80~;\xc9\xdd\x15\xcb\x18\x03V\xd0\xd6\x13K\xd7\xc2T\xc2\xdb\xe6\xb1\xcf[\xdfhl\xb8\xee\x9b5\xc7~\x12\xac\xe8#^\xe1\x15c\xe2\x04>\xf3\xa2\x8b\x0cZ\xa4\xc1#\xbe{g\x83\x95\xe58\xa6f8\x83'

stx = CTransaction()
stx.deserialize(BytesIO(psbt.tx.serialize()))

in_wit0 = CTxInWitness()
in_wit0.scriptWitness.stack = [input_sig0]

in_wit1 = CTxInWitness()
in_wit1.scriptWitness.stack = [input_sig1]

wit = CTxWitness()
wit.vtxinwit = [in_wit0, in_wit1]

wit.serialize()

stx.wit = wit
stx.serialize_with_witness().hex()

#ACCEPTED BY MEMPOOL 

'0200000000010216f79a3163ec37e83c54c6e2e8ff6cd8838e835ef3adc3dfd59627c949ec24090000000000fdfffffff9e21b877c81e1862254fd03941c0391122854284c6bb7fbb11d8acdcaeac0b30000000000fdffffff02d2339700000000002251200b8e7486bc306a0aefb7a7c00af0405542db1acd73eecca82771ccf5fe5eb60f55260000000000001600141399b8a09847a039b0352f58d212c180bab17f770141174c67a5629a05ca885bb25ebc75673ef13a39288ccec71435d22d0539d07aaf7fe5d5d9fb63ee7768a52e7d4c38b4116d35d29af0e98c486051a0cf626f43388301412a90c8307e3bc9dd15cb180356d0d6134bd7c254c2dbe6b1cf5bdf686cb8ee9b35c77e12ace8235ee11563e2043ef3a28b0c5aa4c123be7b678395e538a666388300000000'

In [17]:
# SIGHASH_UNSUPPORTED
psbt = PSBT()
psbt.deserialize(PSBT_IMPLICIT_ALL)

# input 0 
input_pk0 = bip32.get_pubkey_from_path("m/86'/1'/0'/1/2")
input_pk0 = input_pk0[1:] 

psbt.tx.vin[0].prevout.hash = int("0924ec49c92796d5dfc3adf35e838e83d86cffe8e2c6543ce837ec63319af716", 16)
psbt.tx.vin[0].prevout.n = 0
psbt.tx.vin[0].nSequence = 4294967293
psbt.tx.vout[0].nValue = 9909202
psbt.inputs[0].sighash = 0x84
psbt.inputs[0].witness_utxo.nValue = 9909389
psbt.inputs[0].witness_utxo.scriptPubKey = bytes.fromhex("51200b8e7486bc306a0aefb7a7c00af0405542db1acd73eecca82771ccf5fe5eb60f")
psbt.inputs[0].tap_internal_key=bytes.fromhex("029081ece5616aa1cb99319fc4874efee47f08ceba5f6004636c8c43409a5f0a")
psbt.inputs[0].tap_bip32_paths = {input_pk0: (set(), KeyOriginInfo(fpr, [86^H, 1^H, 0^H, 1, 2]))}

psbt.tx.vin.append(CTxIn())
psbt.inputs.append(PartiallySignedInput(0))

#input 1
psbt.inputs[1].witness_utxo = CTxOut()

input_pk1 = bip32.get_pubkey_from_path("m/86'/1'/0'/0/2")
input_pk1 = input_pk1[1:] 

psbt.tx.vin[1].prevout.hash = int("b3c0eacacd8a1db1fbb76b4c2854281291031c9403fd542286e1817c871be2f9", 16)
psbt.tx.vin[1].prevout.n = 0
psbt.tx.vin[1].nSequence = 4294967293
psbt.tx.vout[1].nValue = 9813
psbt.inputs[1].sighash = 0x83
psbt.inputs[1].witness_utxo.nValue = 10000
psbt.inputs[1].witness_utxo.scriptPubKey = bytes.fromhex("512041461489a43cb67e2c64f39427278edd30c03d6d72a4b89986147c428f0cc5f1")
psbt.inputs[1].tap_internal_key=bytes.fromhex("eaf7653443bec65437dba1da52f9ca97e231785af56627e8f9c311cf7300f6de")
psbt.inputs[1].tap_bip32_paths = {input_pk1: (set(), KeyOriginInfo(fpr, [86^H, 1^H, 0^H, 0, 2]))}

psbt.inputs[1].witness_utxo.scriptPubKey.hex()

NEW_PSBT = psbt.serialize()
print(NEW_PSBT)

cHNidP8BAKYCAAAAAhb3mjFj7DfoPFTG4uj/bNiDjoNe863D39WWJ8lJ7CQJAAAAAAD9////+eIbh3yB4YYiVP0DlBwDkRIoVChMa7f7sR2KzcrqwLMAAAAAAP3///8C0jOXAAAAAAAiUSALjnSGvDBqCu+3p8AK8EBVQtsazXPuzKgnccz1/l62D1UmAAAAAAAAFgAUE5m4oJhHoDmwNS9Y0hLBgLqxf3cAAAAAAAEBK400lwAAAAAAIlEgC450hrwwagrvt6fACvBAVULbGs1z7syoJ3HM9f5etg8BAwSEAAAAIRYCkIHs5WFqocuZMZ/Eh07+5H8IzrpfYARjbIxDQJpfChkA9azC/VYAAIABAACAAAAAgAEAAAACAAAAARcgApCB7OVhaqHLmTGfxIdO/uR/CM66X2AEY2yMQ0CaXwoAAQErECcAAAAAAAAiUSBBRhSJpDy2fixk85QnJ47dMMA9bXKkuJmGFHxCjwzF8QEDBIMAAAAhFur3ZTRDvsZUN9uh2lL5ypfiMXha9WYn6PnDEc9zAPbeGQD1rML9VgAAgAEAAIAAAACAAAAAAAIAAAABFyDq92U0Q77GVDfbodpS+cqX4jF4WvVmJ+j5wxHPcwD23gABBSACkIHs5WFqocuZMZ/Eh07+5H8IzrpfYARjbIxDQJpfCiEHApCB7OVhaqHLmTGfxIdO/uR/CM66X2AEY2yMQ0CaXwoZAPWswv1WAACAAQAAgAAAAIABAAAAAgAAAAAA


In [18]:
PSBT_WPKH = "cHNidP8BAHQCAAAAAXoqmXlWwJ+Op/0oGcGph7sU4iv5rc2vIKiXY3Is7uJkAQAAAAD9////AqC7DQAAAAAAGXapFDRKD0jKFQ7CuQOBdmC5tosTpnAmiKx0OCMAAAAAABYAFOs4+puBKPgfJule2wxf+uqDaQ/kAAAAAAABAH0CAAAAAa+/rgZZD3Qf8a9ZtqxGESYzakxKgttVPfb++rc3rDPzAQAAAAD9////AnARAQAAAAAAIgAg/e5EHFblsG0N+CwSTHBwFKXKGWWL4LmFa8oW8e0yWfel9DAAAAAAABYAFDr4QprVlUql7oozyYP9ih6GeZJLAAAAAAEBH6X0MAAAAAAAFgAUOvhCmtWVSqXuijPJg/2KHoZ5kksiBgPuLD2Y6x+TwKGqjlpACbcOt7ROrRXxZm8TawEq1Y0waBj1rML9VAAAgAEAAIAAAACAAQAAAAgAAAAAACICAinsR3JxMe0liKIMRu2pq7fapvSf1Quv5wucWqaWHE7MGPWswv1UAACAAQAAgAAAAIABAAAACgAAAAA="

pubkey = bytes.fromhex("03ee2c3d98eb1f93c0a1aa8e5a4009b70eb7b44ead15f1666f136b012ad58d3068")

sig_sighash1 = b"0E\x02!\x00\xabD\xf3M\xd7\xe8|\x90TY\x12\x97\xa1\x01\xe8P\n\x06A\xd1\xd5\x91\x87\x8d\r#\xcf\x80\x96\xfay\xe8\x02 ]\x12\xd1\x06-\x92^'\xb5{\xdc\xf9\x94\xec\xf32\xad\n\x8eg\xb8\xfe@{\xab!\x01%]\xa62\xaa\x01"
sig_sighash2 = b'0D\x02 o\x86>\xd5\x8b\xb5\xa5\xa2KZ\xcez\xb2\x92\xd0\xce\x04!L_\x8f9\xeb#m3\x9e\xb4\x8d\xc6sK\x02 p\x8d\x95\x0b4B\x02^\xf1nB\xd2\xea\x84b\x14\xc7\x00\x88"\xed\x19o<f}E\xcc\xfa\xc2\xfc\xd3\x02'
sig_sighash3 = b'0D\x02 \x11.vf\xbe\x1bd2\x1cx\x89\xcf\xca(\x03\xb0\xc1\x03\x86\xcb\x08\xe4\xe9\xbf\xef/\x1e\xa1\x93\x02\x01C\x02 .)XC\x991\xa6\x85\xa2\x06\xa4\xf7\xde\xfc\xb7\xce\x0b\xc7\xf6\xd6ov\x8a\xdd\xa9\xb5\xf9\x8f\xb8\x07\x82\xc2\x03'
sig_sighash81 = b"0E\x02!\x00\xde\xae\xfd\x1fg\x96\x9a,\xb9\x0e\xfe\xa9\xc343L\xca=\x9f\xeb4\xcfg\xd62u\xc4c\xa5'0\xd9\x02 rd\x88\x7f s\x93\xd0\x97\xea\xc1@\xc8\xbe\xedu 7w4\x04z\x99.&\xd99\xa1Il/\x82\x81"
sig_sighash82 = b'0E\x02!\x00\xe5\r7m\xa2\x1a\xb4\x89\xd48k\x14\xeb\xd0\xa9\xcc\x00\x17\x9ch\x8b\x16\xb5\x9d&\xab\x94md9\x929\x02 "\x159\xdc\xa3\x06\x06\x9cR\n\xf1\x9a\xfb^\xde)\x1a\xe9\x1e\x07S\x96\xedARN\xfeY\xa4\xc1A\xd4\x82'
sig_sighash83 = b'0D\x02 \x07q\xb3\xe4\x05\xa3|\xd4\xaa$\x95\x1c\x08\x8d~L7\t:|\xddp7\xa7h\x81\x14\xd5$V\x03v\x02 @\xff\xf9\xbc\xd0|\x00\xfa\x91-}\x1e\xed\x04\x0e\xcc\x9d\xd4\xe4NM\\\xf6\xef\x9a\x94\xaf\x83l\xd8\x7f\xdd\x83'

def sign(sig):

    psbt = PSBT()
    psbt.deserialize(PSBT_WPKH)
    
    psbt.inputs[0].partial_sigs[pubkey] = sig

    stx = CTransaction()
    stx.deserialize(BytesIO(psbt.tx.serialize()))

    in_wit0 = CTxInWitness()
    in_wit0.scriptWitness.stack = [sig, pubkey]

    wit = CTxWitness()
    wit.vtxinwit = [in_wit0]
    wit.serialize()

    stx.wit = wit
    print(stx.serialize_with_witness().hex())

#ACCEPTED BY MEMPOOL


print("SIGHASH1")
sign(sig_sighash1)
print("SIGHASH2")
sign(sig_sighash2)
print("SIGHASH3")
sign(sig_sighash3)
print("SIGHASH81")
sign(sig_sighash81)
print("SIGHASH82")
sign(sig_sighash82)
print("SIGHASH83")
sign(sig_sighash83)

SIGHASH1
020000000001017a2a997956c09f8ea7fd2819c1a987bb14e22bf9adcdaf20a89763722ceee2640100000000fdffffff02a0bb0d00000000001976a914344a0f48ca150ec2b903817660b9b68b13a6702688ac7438230000000000160014eb38fa9b8128f81f26e95edb0c5ffaea83690fe402483045022100ab44f34dd7e87c9054591297a101e8500a0641d1d591878d0d23cf8096fa79e802205d12d1062d925e27b57bdcf994ecf332ad0a8e67b8fe407bab2101255da632aa012103ee2c3d98eb1f93c0a1aa8e5a4009b70eb7b44ead15f1666f136b012ad58d306800000000
SIGHASH2
020000000001017a2a997956c09f8ea7fd2819c1a987bb14e22bf9adcdaf20a89763722ceee2640100000000fdffffff02a0bb0d00000000001976a914344a0f48ca150ec2b903817660b9b68b13a6702688ac7438230000000000160014eb38fa9b8128f81f26e95edb0c5ffaea83690fe40247304402206f863ed58bb5a5a24b5ace7ab292d0ce04214c5f8f39eb236d339eb48dc6734b0220708d950b3442025ef16e42d2ea846214c7008822ed196f3c667d45ccfac2fcd3022103ee2c3d98eb1f93c0a1aa8e5a4009b70eb7b44ead15f1666f136b012ad58d306800000000
SIGHASH3
020000000001017a2a997956c09f8ea7fd2819c1a987bb14e22bf9adcdaf20a897637